In [16]:
#importing dependencies:
%matplotlib inline

#importing the python libraries:
import math, time, random, datetime

#data manipulation:
import numpy as np
import pandas as pd

#for visualization:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("seaborn-whitegrid")

#preprocessing
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, label_binarize

#import ML algorithms
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier 
from sklearn.tree import DecisionTreeClassifier

In [ ]:
#importing train and test data
train = pd.read_csv("../ML/Titanic")